# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [103]:
%load_ext autoreload
%autoreload 2

In [104]:
pip install numpy==1.22.1

Note: you may need to restart the kernel to use updated packages.


In [105]:
pip install pandas==1.4.2

Note: you may need to restart the kernel to use updated packages.


In [106]:
# pip install --upgrade scikit-learn


In [107]:
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [108]:
import warnings

warnings.filterwarnings('ignore', message="The sklearn.metrics.classification module", category=FutureWarning)
warnings.filterwarnings('ignore', message=".*title_format is deprecated. Please use title instead.*")
warnings.filterwarnings('ignore', message="optional dependency `torch` is not available. - skipping import of NN models.")
warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

In [109]:
# import numpy as np
# np.NaN = np.nan



In [110]:
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)

1.22.1
1.4.2


In [111]:

# Standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment
)
import ta_lib.core.api as dataset
import ta_lib.eda.api as ta_analysis

# Set environment variables
os.environ['TA_DEBUG'] = "False"
os.environ['TA_ALLOW_EXCEPTIONS'] = "True"

# Initialization
initialize_environment(debug=False, hide_warnings=True)


## Utility functions

In [112]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
print(dataset.list_datasets(context))

['/raw/sod', '/raw/spd', '/cleaned/oppty_data', '/cleaned/prod_data']


## Load and clean data

In [113]:
data = dict()

for i in dataset.list_datasets(context):
    if '/cleaned/' in i:
        dataset_name = i.replace('/cleaned/','')
        key_ = dataset_name+'_df'
        data[key_] = dataset.load_dataset(context,i)
        
        # Standardize column names
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)

        # Data type changes
        if key_ == 'oppty_data_df':
            # data[key_]['transition from timestamp'] = pd.to_datetime(data[key_]['transition from timestamp'],format='%Y%m%d%H%M%S')
            # data[key_]['transition to timestamp'] = pd.to_datetime(data[key_]['transition to timestamp'],format='%Y%m%d%H%M%S')
            oppty_data_df=data["oppty_data_df"]
        elif key_ == 'prod_data_df':
            # data[key_]['snapshot time'] = pd.to_datetime(data[key_]['snapshot time'], format='%Y%m%d%H%M%S')
            # data[key_]['decision date'] = pd.to_datetime(data[key_]['decision date'], unit='D', origin='1899-12-30')
            prod_data_df=data["prod_data_df"]
        
        # Drop duplicates
        data[key_].drop_duplicates(inplace=True)
        
        # Save processed data
        dataset.save_dataset(context, data[key_], 'cleaned/'+dataset_name)

In [114]:
for k,v in data.items():
    print(k)

oppty_data_df
prod_data_df


In [115]:
oppty_data_df.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,Stage 3,2015-02-13 05:00:02,Stage 3,2015-02-11 13:00:01,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,Stage 3,2015-02-18 21:00:07,Stage 3,2015-02-13 05:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-02-18 21:00:07,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-03-04 21:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4


In [116]:
prod_data_df.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product segment name,product status,product $,product quantity,decision date,snapshot time
0,0,0,0,0,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-11-25 05:00:02
1,1,1,1,1,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-10-25 05:00:02
2,2,2,2,2,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-05 05:00:03
3,3,3,3,3,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-01 05:00:03
4,4,4,4,4,1361,6669,Product Segment NA,Pending,0,1,2015-08-31,2015-05-25 05:00:03


In [117]:
for key,df in data.items():
    print(key+" dataframe")
    print(data[key].isnull().sum())
    print()

oppty_data_df dataframe
unnamed 0                        0
unnamed 0                        0
unnamed 0.2                      0
unnamed 0.1                      0
opportunity id                   0
transition to stage              0
transition to timestamp          0
transition from stage name       0
transition from timestamp     5250
customer name                    0
risk status                      0
creation date                    0
decision date                    0
opportunity status               0
opportunity type                 0
core consumption market          0
core product segment             0
core sales segment               0
geography                        0
core product application         0
dtype: int64

prod_data_df dataframe
unnamed 0               0
unnamed 0               0
unnamed 0.2             0
unnamed 0.1             0
opportunity id          0
product id              0
product segment name    0
product status          0
product $               0
produ

In [118]:
oppty_data_df['opportunity status'].value_counts()

Open           104119
Closed/Won       3558
Closed/Lost       526
Name: opportunity status, dtype: int64

In [119]:
## Removing rows of each opportunity after the first occurence of it's status as 'Closed/won' or 'Closed/lost'

endIndex=0

# Process the DataFrame
for i in oppty_data_df['opportunity id'].unique():

    # Initialize index variable to store the first occurrence of 'Closed/Won' or 'Closed/Lost'
    index = 0
    # Initialize flag variable to know the first occurence of 'Closed/Won' or 'Closed/Lost'
    flag=0
    # Iterate over the 'opportunity status' column to find the first occurrence of 'Closed/Won' or 'Closed/Lost'
    for j in oppty_data_df[oppty_data_df['opportunity id']==i]['opportunity status']:
        # For every occurence of 'opportunity status', increment the index
        index+=1
        # Check for the first occurence of 'Closed/Won' or 'Closed/Lost'
        if j in ['Closed/Won', 'Closed/Lost']:
            # Change flag value to 1 and break the loop
            flag=1
            break
    # If flag=1 then drop rows for the same opportunity id after status is "Closed/Won" or "Closed/Lost"
    if flag==1:
        oppty_data_df.drop(oppty_data_df.index[index+endIndex:endIndex+len(oppty_data_df[
                                                    oppty_data_df['opportunity id']==i])]
                                                    ,inplace=True,axis=0)
        # Reassign flag to 0
        flag=0
        # Increment endIndex with lenght of each opportunity id
    endIndex+=len(oppty_data_df[oppty_data_df['opportunity id']==i])

# Display final information about the DataFrame
oppty_data_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,Stage 3,2015-02-13 05:00:02,Stage 3,2015-02-11 13:00:01,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,Stage 3,2015-02-18 21:00:07,Stage 3,2015-02-13 05:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-02-18 21:00:07,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-03-04 21:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108198,108198,108198,313566,313566,75083,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-06-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108199,108199,108199,313567,313567,75085,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108200,108200,108200,313568,313568,75085,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108201,108201,108201,313569,313569,75086,Stage 6,2016-10-23 21:00:04,Stage 0,NaN,Customer 306,No Risk,2016-10-23,2017-01-31,Open,Design,Core Market 16,Core Prd Seg 3,Sales Segment 13,Geo 2,Prd App 14


In [120]:
#dropping null values from product_df
print(prod_data_df.isnull().sum())
print(prod_data_df['product status'].value_counts())
prod_data_df.dropna(axis=0,inplace=True)
print(prod_data_df.isnull().sum())

unnamed 0               0
unnamed 0               0
unnamed 0.2             0
unnamed 0.1             0
opportunity id          0
product id              0
product segment name    0
product status          0
product $               0
product quantity        0
decision date           0
snapshot time           0
dtype: int64
Win Approved     79749
Pending          46587
Deleted           4530
Cancelled         4123
Win Submitted     3917
Lost              2305
Name: product status, dtype: int64
unnamed 0               0
unnamed 0               0
unnamed 0.2             0
unnamed 0.1             0
opportunity id          0
product id              0
product segment name    0
product status          0
product $               0
product quantity        0
decision date           0
snapshot time           0
dtype: int64


In [121]:
prod_data_df
oppty_data_df


,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,Stage 3,2015-02-13 05:00:02,Stage 3,2015-02-11 13:00:01,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,Stage 3,2015-02-18 21:00:07,Stage 3,2015-02-13 05:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-02-18 21:00:07,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-03-04 21:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108198,108198,108198,313566,313566,75083,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-06-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108199,108199,108199,313567,313567,75085,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108200,108200,108200,313568,313568,75085,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108201,108201,108201,313569,313569,75086,Stage 6,2016-10-23 21:00:04,Stage 0,NaN,Customer 306,No Risk,2016-10-23,2017-01-31,Open,Design,Core Market 16,Core Prd Seg 3,Sales Segment 13,Geo 2,Prd App 14


In [122]:
print(oppty_data_df.isnull().sum())
print(oppty_data_df['risk status'].value_counts())
oppty_data_df['risk status'].fillna('None',inplace=True)
oppty_data_df['opportunity type'].fillna('None',inplace=True)
oppty_data_df['opportunity status'].fillna('Open',inplace=True)
print(oppty_data_df.isnull().sum())

unnamed 0                        0
unnamed 0                        0
unnamed 0.2                      0
unnamed 0.1                      0
opportunity id                   0
transition to stage              0
transition to timestamp          0
transition from stage name       0
transition from timestamp     5250
customer name                    0
risk status                      0
creation date                    0
decision date                    0
opportunity status               0
opportunity type                 0
core consumption market          0
core product segment             0
core sales segment               0
geography                        0
core product application         0
dtype: int64
None       80175
No Risk    27640
Others       291
Pricing       97
Name: risk status, dtype: int64
unnamed 0                        0
unnamed 0                        0
unnamed 0.2                      0
unnamed 0.1                      0
opportunity id                   0
transition t

In [123]:
oppo_dummy=oppty_data_df.drop_duplicates(subset='opportunity id',keep='first')
oppo_dummy

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
6,6,6,102,102,19,Stage 4,2015-02-11 13:00:01,Stage 0,NaN,Customer 216,None,2015-02-11,2015-01-01,Open,None,Core Market 3,Core Prd Seg 3,Sales Segment 1,Geo NA,Prd App 13
30,30,30,214,214,46,Stage 2,2015-02-11 13:00:01,Stage 0,NaN,Customer 108,None,2015-02-11,2015-04-17,Open,None,Core Market 3,Core Prd Seg 3,Sales Segment 1,Geo NA,Prd App 4
51,51,51,322,322,47,Stage 6,2015-02-11 13:00:01,Stage 0,NaN,Customer 117,None,2015-02-11,2015-04-30,Open,None,Core Market 3,Core Prd Seg 4,Sales Segment 1,Geo NA,Prd App 15
57,57,57,425,425,93,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 286,None,2015-02-11,2015-02-28,Open,None,Core Market 3,Core Prd Seg 3,Sales Segment 1,Geo NA,Prd App 13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108193,108193,108193,313561,313561,75080,Stage 6,2016-10-22 05:00:03,Stage 0,NaN,Customer 7035,No Risk,2016-10-22,2016-02-26,Open,Design,Core Market 16,Core Prd Seg 3,Sales Segment 1,Geo 3,Prd App 13
108195,108195,108195,313563,313563,75082,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-03-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108197,108197,108197,313565,313565,75083,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-06-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108199,108199,108199,313567,313567,75085,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13


In [124]:
oppo_dummy.isnull().sum()

unnamed 0                        0
unnamed 0                        0
unnamed 0.2                      0
unnamed 0.1                      0
opportunity id                   0
transition to stage              0
transition to timestamp          0
transition from stage name       0
transition from timestamp     5250
customer name                    0
risk status                      0
creation date                    0
decision date                    0
opportunity status               0
opportunity type                 0
core consumption market          0
core product segment             0
core sales segment               0
geography                        0
core product application         0
dtype: int64

In [125]:
# From above we can conclude that for all opportunities the first value of 'transition from stage name'
# column is null, so let us replace it with 'Stage 0'
oppty_data_df['transition from stage name']=oppty_data_df['transition from stage name'].fillna('Stage 0')

In [126]:
oppty_data_df.isnull().sum()

unnamed 0                        0
unnamed 0                        0
unnamed 0.2                      0
unnamed 0.1                      0
opportunity id                   0
transition to stage              0
transition to timestamp          0
transition from stage name       0
transition from timestamp     5250
customer name                    0
risk status                      0
creation date                    0
decision date                    0
opportunity status               0
opportunity type                 0
core consumption market          0
core product segment             0
core sales segment               0
geography                        0
core product application         0
dtype: int64

In [127]:
dataset.save_dataset(context, oppty_data_df, '/cleaned/oppty_data', index = False)
dataset.save_dataset(context, prod_data_df, '/cleaned/prod_data', index = False)

In [128]:
oppty_data_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,Stage 3,2015-02-11 13:00:01,Stage 0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,Stage 3,2015-02-13 05:00:02,Stage 3,2015-02-11 13:00:01,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,Stage 3,2015-02-18 21:00:07,Stage 3,2015-02-13 05:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-02-18 21:00:07,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,Stage 3,2015-03-04 21:00:02,Stage 3,2015-03-04 21:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108198,108198,108198,313566,313566,75083,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-06-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108199,108199,108199,313567,313567,75085,Stage 6,2016-10-23 13:00:03,Stage 0,NaN,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108200,108200,108200,313568,313568,75085,Stage 6,2016-10-25 07:51:57,Stage 6,2016-10-23 13:00:03,Customer 141,No Risk,2016-10-23,2017-12-25,Open,Deal,Core Market 19,Core Prd Seg 3,Sales Segment 8,Geo 4,Prd App 13
108201,108201,108201,313569,313569,75086,Stage 6,2016-10-23 21:00:04,Stage 0,NaN,Customer 306,No Risk,2016-10-23,2017-01-31,Open,Design,Core Market 16,Core Prd Seg 3,Sales Segment 13,Geo 2,Prd App 14


In [129]:
prod_data_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product segment name,product status,product $,product quantity,decision date,snapshot time
0,0,0,0,0,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-11-25 05:00:02
1,1,1,1,1,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-10-25 05:00:02
2,2,2,2,2,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-05 05:00:03
3,3,3,3,3,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-01 05:00:03
4,4,4,4,4,1361,6669,Product Segment NA,Pending,0,1,2015-08-31,2015-05-25 05:00:03
...,...,...,...,...,...,...,...,...,...,...,...,...
141206,141206,141206,141254,142426,28546,45808,Product Segment NA,Pending,327600,2800,2016-07-27,2016-01-10 05:00:05
141207,141207,141207,141255,142427,28546,45808,Product Segment NA,Pending,327600,2800,2016-07-27,2016-01-09 13:00:03
141208,141208,141208,141256,142428,28550,45810,Product Segment NA,Pending,192000,1000,2016-01-29,2016-01-09 13:00:03
141209,141209,141209,141257,142429,28551,45792,Product Segment NA,Pending,160000,2500,2016-03-31,2016-01-10 05:00:05


In [130]:
prod_data_df.drop_duplicates(inplace=True)
prod_data_df=prod_data_df.reset_index(drop=True)
prod_data_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product segment name,product status,product $,product quantity,decision date,snapshot time
0,0,0,0,0,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-11-25 05:00:02
1,1,1,1,1,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-10-25 05:00:02
2,2,2,2,2,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-05 05:00:03
3,3,3,3,3,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-01 05:00:03
4,4,4,4,4,1361,6669,Product Segment NA,Pending,0,1,2015-08-31,2015-05-25 05:00:03
...,...,...,...,...,...,...,...,...,...,...,...,...
141206,141206,141206,141254,142426,28546,45808,Product Segment NA,Pending,327600,2800,2016-07-27,2016-01-10 05:00:05
141207,141207,141207,141255,142427,28546,45808,Product Segment NA,Pending,327600,2800,2016-07-27,2016-01-09 13:00:03
141208,141208,141208,141256,142428,28550,45810,Product Segment NA,Pending,192000,1000,2016-01-29,2016-01-09 13:00:03
141209,141209,141209,141257,142429,28551,45792,Product Segment NA,Pending,160000,2500,2016-03-31,2016-01-10 05:00:05


In [131]:
pd.set_option('display.max_columns', None)

In [132]:
oppty_data_df.columns

Index(['unnamed 0', 'unnamed 0', 'unnamed 0.2', 'unnamed 0.1',
       'opportunity id', 'transition to stage', 'transition to timestamp',
       'transition from stage name', 'transition from timestamp',
       'customer name', 'risk status', 'creation date', 'decision date',
       'opportunity status', 'opportunity type', 'core consumption market',
       'core product segment', 'core sales segment', 'geography',
       'core product application'],
      dtype='object')

In [133]:
opportunities=list(prod_data_df['opportunity id'].unique())
opportunities

[1361,
 1363,
 1364,
 5,
 19,
 46,
 47,
 93,
 99,
 105,
 114,
 5104,
 138,
 144,
 151,
 173,
 227,
 260,
 263,
 288,
 303,
 325,
 331,
 340,
 367,
 369,
 380,
 423,
 432,
 2331,
 2381,
 2426,
 2431,
 2432,
 2437,
 2446,
 2450,
 2453,
 2491,
 2508,
 2552,
 2564,
 2570,
 2581,
 2596,
 2603,
 2616,
 2617,
 2634,
 2636,
 2658,
 2679,
 2724,
 501,
 517,
 526,
 541,
 546,
 558,
 587,
 598,
 571,
 659,
 678,
 686,
 689,
 698,
 706,
 713,
 734,
 756,
 768,
 778,
 801,
 804,
 819,
 821,
 838,
 846,
 850,
 892,
 1374,
 1375,
 1383,
 930,
 935,
 1464,
 973,
 1473,
 986,
 988,
 1504,
 1000,
 1020,
 1080,
 1096,
 1547,
 1134,
 1589,
 1599,
 1602,
 1620,
 1630,
 1643,
 1651,
 1222,
 1664,
 1671,
 1267,
 1273,
 1304,
 1719,
 1730,
 1742,
 1775,
 1360,
 42745,
 1809,
 1852,
 1881,
 1896,
 1900,
 1929,
 1945,
 1975,
 1986,
 2013,
 2022,
 2044,
 2070,
 2114,
 2134,
 2143,
 2185,
 2212,
 2225,
 2228,
 2240,
 4283,
 4290,
 2773,
 2789,
 2791,
 4299,
 2831,
 4341,
 4348,
 4367,
 4381,
 4417,
 4453,
 4482,


In [134]:
oppty_df_new=oppty_data_df[oppty_data_df['opportunity id'].isin(opportunities)]

In [135]:
for col in ['transition to stage', 'transition from stage name']:
                temp=[]
                for i in oppty_df_new[col]:
                    if type(i)==str:
                        temp.append(int(i[-1]))
                oppty_df_new[col]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,Customer 83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4


In [136]:
oppty_df_new[oppty_df_new['customer name']=='Customer NA'].size

120

In [137]:
temp=[]
for i in oppty_df_new["customer name"]:
    if i!='Customer NA':
        cust_no = int(i.split(" ")[-1])
        temp.append(cust_no)
    else:
        temp.append(-1)
oppty_df_new["customer name"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,Core Market 11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4


In [138]:
temp=[]
for i in oppty_df_new["core consumption market"]:
    market_no = int(i.split(" ")[-1])
    temp.append(market_no)
oppty_df_new["core consumption market"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,Core Prd Seg 3,Sales Segment 8,Geo NA,Prd App 4


In [139]:
temp=[]
for i in oppty_df_new["core product segment"]:
    seg_no = int(i.split(" ")[-1])
    temp.append(seg_no)
oppty_df_new["core product segment"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,3,Sales Segment 8,Geo NA,Prd App 4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,3,Sales Segment 8,Geo NA,Prd App 4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,Sales Segment 8,Geo NA,Prd App 4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,3,Sales Segment 8,Geo NA,Prd App 4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,Sales Segment 8,Geo NA,Prd App 4


In [140]:
temp=[]
for i in oppty_df_new["core sales segment"]:
    seg_no = int(i.split(" ")[-1])
    temp.append(seg_no)
oppty_df_new["core sales segment"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,Prd App 4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,Prd App 4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,Prd App 4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,Prd App 4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,Prd App 4


In [141]:
sum=0
for i in oppty_df_new["core product application"]:
    appli_no = int(i.split(" ")[-1])
    if appli_no=='NA':
        sum+=1
sum

0

In [142]:
temp=[]
for i in oppty_df_new["core product application"]:
    appli_no = int(i.split(" ")[-1])
    temp.append(appli_no)
oppty_df_new["core product application"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,Geo NA,4


In [143]:
temp=[]
for i in oppty_df_new["geography"]:
    if i!='Geo NA' and i!='Geo 1':
        geo_no = int(i.split(" ")[-1])
        temp.append(geo_no)
    else:
        temp.append(-1)
oppty_df_new["geography"]=temp
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4


In [144]:
oppty_df_new['risk status'].replace('None',0,inplace=True)
oppty_df_new['risk status'].replace('No Risk',1,inplace=True)
oppty_df_new['risk status'].replace('Others',2,inplace=True)
oppty_df_new['risk status'].replace('Pricing',3,inplace=True)

In [145]:
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,opportunity status,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Open,None,11,3,8,-1,4


In [146]:
oppty_df_new['opportunity status'].replace(['Open','Closed/Lost'],0,inplace=True)
oppty_df_new['opportunity status'].replace('Closed/Won',1,inplace=True)

# Replace column name
oppty_df_new.rename(columns={'opportunity status': 'win'}, inplace=True)

In [147]:
last_status = oppty_df_new.groupby('opportunity id')['win'].transform('last')

# Assign the last status value to all rows of each id
oppty_df_new['win'] = last_status

In [148]:
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4


In [149]:
oppty_df_new['opportunity type'].replace('Deal',0,inplace=True)
oppty_df_new['opportunity type'].replace('Design',1,inplace=True)
oppty_df_new['opportunity type'].replace('None',2,inplace=True)

In [150]:
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaN,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4


In [151]:
oppty_df_new['decision date'].replace('2215-03-20 00:00:00','2015-03-20 00:00:00',inplace=True)

In [152]:
oppty_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57260 entries, 0 to 95603
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   unnamed 0                   57260 non-null  int64 
 1   unnamed 0                   57260 non-null  int64 
 2   unnamed 0.2                 57260 non-null  int64 
 3   unnamed 0.1                 57260 non-null  int64 
 4   opportunity id              57260 non-null  int64 
 5   transition to stage         57260 non-null  int64 
 6   transition to timestamp     57260 non-null  object
 7   transition from stage name  57260 non-null  int64 
 8   transition from timestamp   54722 non-null  object
 9   customer name               57260 non-null  int64 
 10  risk status                 57260 non-null  object
 11  creation date               57260 non-null  object
 12  decision date               57260 non-null  object
 13  win                         57260 non-null  ob

In [153]:
oppty_df_new['transition to timestamp']=pd.to_datetime(oppty_df_new['transition to timestamp'])
oppty_df_new['transition from timestamp']=pd.to_datetime(oppty_df_new['transition from timestamp'])
oppty_df_new['creation date']=pd.to_datetime(oppty_df_new['creation date'])
oppty_df_new['decision date']=pd.to_datetime(oppty_df_new['decision date'])

In [154]:
oppty_df_new

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaT,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87556,87556,87556,276303,276303,42789,6,2016-06-01 05:00:03,6,2016-05-25 07:21:17,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5
87557,87557,87557,276304,276304,42789,6,2016-06-06 08:05:32,6,2016-06-01 05:00:03,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5
87558,87558,87558,276305,276305,42789,6,2016-06-10 07:13:44,6,2016-06-06 08:05:32,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5
87559,87559,87559,276306,276306,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5


In [155]:
oppty_df_new['no of days to decision']=oppty_df_new['decision date'] - oppty_df_new['creation date']
oppty_df_new['no of days to decision']=oppty_df_new['no of days to decision'].dt.days
oppty_df_new.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application,no of days to decision
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaT,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29


In [156]:
oppty_df_new['transition days']=oppty_df_new['transition to timestamp']-oppty_df_new['transition from timestamp']
oppty_df_new['transition days']=oppty_df_new['transition days'].dt.days
oppty_df_new

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application,no of days to decision,transition days
0,0,0,0,0,5,3,2015-02-11 13:00:01,0,NaT,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,NaN
1,1,1,1,1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,1.0
2,2,2,2,2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,5.0
3,3,3,3,3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,13.0
4,4,4,4,4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87556,87556,87556,276303,276303,42789,6,2016-06-01 05:00:03,6,2016-05-25 07:21:17,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,6.0
87557,87557,87557,276304,276304,42789,6,2016-06-06 08:05:32,6,2016-06-01 05:00:03,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,5.0
87558,87558,87558,276305,276305,42789,6,2016-06-10 07:13:44,6,2016-06-06 08:05:32,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,3.0
87559,87559,87559,276306,276306,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,2.0


In [157]:
oppty_df_new['total transition days']=oppty_df_new.groupby('opportunity id')['transition days'].transform('sum')

In [158]:
oppty_df_new['opportunity id'].nunique()

2538

In [159]:
prod_data_df.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product segment name,product status,product $,product quantity,decision date,snapshot time
0,0,0,0,0,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-11-25 05:00:02
1,1,1,1,1,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2015-10-25 05:00:02
2,2,2,2,2,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-05 05:00:03
3,3,3,3,3,1361,6669,Product Segment NA,Win Approved,0,1,2015-08-12,2016-01-01 05:00:03
4,4,4,4,4,1361,6669,Product Segment NA,Pending,0,1,2015-08-31,2015-05-25 05:00:03


In [160]:
prod_data_df['product segment name'].value_counts()

Product Segment NA    141211
Name: product segment name, dtype: int64

In [161]:
prod_data_df=prod_data_df.drop(columns='product segment name',axis=1)

In [162]:
prod_data_df.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
0,0,0,0,0,1361,6669,Win Approved,0,1,2015-08-12,2015-11-25 05:00:02
1,1,1,1,1,1361,6669,Win Approved,0,1,2015-08-12,2015-10-25 05:00:02
2,2,2,2,2,1361,6669,Win Approved,0,1,2015-08-12,2016-01-05 05:00:03
3,3,3,3,3,1361,6669,Win Approved,0,1,2015-08-12,2016-01-01 05:00:03
4,4,4,4,4,1361,6669,Pending,0,1,2015-08-31,2015-05-25 05:00:03


In [163]:
prod_data_df=prod_data_df.sort_values(by=['opportunity id','product id','decision date','snapshot time'])

In [164]:
prod_data_df=prod_data_df.reset_index(drop=True)

In [165]:
prod_data_df[prod_data_df['opportunity id']==19]

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
63,518,518,518,526,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
64,457,457,457,465,19,16377,Pending,0,400000,2015-08-14,2015-05-15 05:00:03
65,328,328,328,336,19,16377,Pending,0,400000,2015-08-14,2015-05-20 05:00:03
66,390,390,390,398,19,16377,Pending,0,400000,2015-08-14,2015-05-25 05:00:03
67,375,375,375,383,19,16377,Pending,0,400000,2015-08-14,2015-06-01 14:02:36
...,...,...,...,...,...,...,...,...,...,...,...
302,324,324,324,332,19,17371,Win Approved,0,400000,2015-08-18,2015-12-20 05:00:02
303,511,511,511,519,19,17371,Win Approved,0,400000,2015-08-18,2015-12-25 05:00:03
304,471,471,471,479,19,17371,Win Approved,0,400000,2015-08-18,2016-01-01 05:00:03
305,306,306,306,314,19,17371,Win Approved,0,400000,2015-08-18,2016-01-05 05:00:03


In [166]:
prod_data_df['product status'].value_counts()

Win Approved     79749
Pending          46587
Deleted           4530
Cancelled         4123
Win Submitted     3917
Lost              2305
Name: product status, dtype: int64

In [167]:
prod_data_df['product status'].replace(['Win Approved','Win Submitted'],1,inplace=True)
prod_data_df['product status'].replace(['Pending','Deleted','Cancelled','Lost'],0,inplace=True)

In [168]:
result_product_df = pd.DataFrame(columns=prod_data_df.columns)

# Iterate over each unique 'opportunity id' and 'product id'
for opp_id in prod_data_df['opportunity id'].unique():
    for prod_id in prod_data_df[prod_data_df['opportunity id'] == opp_id]['product id'].unique():
        temp = prod_data_df[(prod_data_df['opportunity id'] == opp_id) & (prod_data_df['product id'] == prod_id)].copy()

        # Find the first occurrence of 'Win Submitted'
        first_win_index = temp[temp['product status'] == 0].index.min()

        # If 'Win Submitted' is found, keep rows up to that index, otherwise keep all
        if not pd.isna(first_win_index):
            temp = temp.loc[:first_win_index]


        # Append the filtered rows to the result DataFrame
        result_product_df  = pd.concat([result_product_df, temp])

# Reset the index of the result DataFrame
result_product_df.reset_index(drop=True, inplace=True)
result_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
0,241,241,241,245,5,404,Pending,230400,1200,2015-03-12,2015-02-11 21:00:01
1,273,273,273,279,5,404,Pending,230400,1200,2015-03-12,2015-02-13 05:00:02
2,243,243,243,247,5,404,Pending,230400,1200,2015-03-12,2015-02-18 21:00:07
3,253,253,253,258,5,404,Pending,230400,1200,2015-03-12,2015-03-04 21:00:02
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01
...,...,...,...,...,...,...,...,...,...,...,...
141206,105081,105081,105129,106301,56117,25835,Win Approved,7605,65,2015-11-23,2015-12-20 05:00:02
141207,105068,105068,105116,106288,56117,25835,Win Approved,7605,65,2015-11-23,2015-12-25 05:00:03
141208,105072,105072,105120,106292,56117,25835,Win Approved,7605,65,2015-11-23,2016-01-01 05:00:03
141209,105086,105086,105134,106306,56117,25835,Win Approved,7605,65,2015-11-23,2016-01-05 05:00:03


In [169]:
filtered_product_df=result_product_df.drop_duplicates(subset=['opportunity id', 'product id', 'product status', 'product $',
       'product quantity'],keep='first')
filtered_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
0,241,241,241,245,5,404,Pending,230400,1200,2015-03-12,2015-02-11 21:00:01
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01
63,518,518,518,526,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02
112,435,435,435,443,19,16378,Pending,0,200000,2015-08-14,2015-05-11 05:00:04
...,...,...,...,...,...,...,...,...,...,...,...
141153,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02
141164,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03
141165,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03
141177,105065,105065,105113,106285,56117,25835,Pending,7605,65,2015-08-28,2015-07-24 05:00:03


In [170]:
filtered_product_df[filtered_product_df['opportunity id']==19]

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
63,518,518,518,526,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02
112,435,435,435,443,19,16378,Pending,0,200000,2015-08-14,2015-05-11 05:00:04
132,468,468,468,476,19,16378,Win Approved,0,200000,2015-08-18,2015-08-18 13:00:02
160,464,464,464,472,19,16380,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
180,364,364,364,372,19,16380,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02
208,436,436,436,444,19,16381,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
228,439,439,439,447,19,16381,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02
258,378,378,378,386,19,17371,Pending,0,400000,2015-08-14,2015-05-19 05:00:02
277,305,305,305,313,19,17371,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02


In [171]:
temp_product_df=filtered_product_df.drop_duplicates(subset=['opportunity id','product id','product $','product quantity'],keep='last')

temp_product_df['product $'] = temp_product_df['product $'].astype(int)
temp_product_df['product quantity'] = temp_product_df['product quantity'].astype(int)

temp_product_df['opportunity average price']=temp_product_df.groupby('opportunity id')['product $'].transform('mean')
temp_product_df['opportunity total quantity']=temp_product_df.groupby('opportunity id')['product quantity'].transform('sum')
temp_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
132,468,468,468,476,19,16378,Win Approved,0,200000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
180,364,364,364,372,19,16380,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
228,439,439,439,447,19,16381,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141142,134983,134983,135031,136203,42778,38156,Pending,6000000,50000,2016-01-31,2015-11-15 13:00:03,6000000.0,50000
141153,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02,75600.0,25200
141164,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03,33000.0,11000
141165,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03,33000.0,11000


In [172]:
temp_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
132,468,468,468,476,19,16378,Win Approved,0,200000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
180,364,364,364,372,19,16380,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
228,439,439,439,447,19,16381,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141142,134983,134983,135031,136203,42778,38156,Pending,6000000,50000,2016-01-31,2015-11-15 13:00:03,6000000.0,50000
141153,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02,75600.0,25200
141164,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03,33000.0,11000
141165,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03,33000.0,11000


In [173]:
temp_product_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5166 entries, 4 to 141200
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   unnamed 0                   5166 non-null   object 
 1   unnamed 0                   5166 non-null   object 
 2   unnamed 0.2                 5166 non-null   object 
 3   unnamed 0.1                 5166 non-null   object 
 4   opportunity id              5166 non-null   object 
 5   product id                  5166 non-null   object 
 6   product status              5166 non-null   object 
 7   product $                   5166 non-null   int64  
 8   product quantity            5166 non-null   int64  
 9   decision date               5166 non-null   object 
 10  snapshot time               5166 non-null   object 
 11  opportunity average price   5166 non-null   float64
 12  opportunity total quantity  5166 non-null   int64  
dtypes: float64(1), int64(3), object

In [175]:
temp_product_df['no of products']=temp_product_df.groupby('opportunity id')['product id'].transform('count')
temp_product_df['no of products lost']=temp_product_df.groupby('opportunity id')['product status'].transform('count')
temp_product_df['no of products won']=temp_product_df['no of products']-temp_product_df['no of products lost']
temp_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200,1,1,0
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
132,468,468,468,476,19,16378,Win Approved,0,200000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
180,364,364,364,372,19,16380,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
228,439,439,439,447,19,16381,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141142,134983,134983,135031,136203,42778,38156,Pending,6000000,50000,2016-01-31,2015-11-15 13:00:03,6000000.0,50000,1,1,0
141153,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02,75600.0,25200,1,1,0
141164,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03,33000.0,11000,2,2,0
141165,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03,33000.0,11000,2,2,0


In [176]:
temp_product_df[temp_product_df['no of products won']>0]

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won


In [177]:
temp_product_df

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200,1,1,0
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
132,468,468,468,476,19,16378,Win Approved,0,200000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
180,364,364,364,372,19,16380,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
228,439,439,439,447,19,16381,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141142,134983,134983,135031,136203,42778,38156,Pending,6000000,50000,2016-01-31,2015-11-15 13:00:03,6000000.0,50000,1,1,0
141153,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02,75600.0,25200,1,1,0
141164,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03,33000.0,11000,2,2,0
141165,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03,33000.0,11000,2,2,0


In [178]:
temp_product_df=temp_product_df[['opportunity id',
       'opportunity average price', 'opportunity total quantity',
       'no of products', 'no of products lost', 'no of products won']]
temp_product_df=temp_product_df.drop_duplicates()
temp_product_df.reset_index(drop=True,inplace=True)

In [179]:
temp_product_df.head()

,opportunity id,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won
0,5,230400.0,1200,1,1,0
1,19,0.0,1800000,5,5,0
2,46,840125.0,11000,4,4,0
3,47,1400000.0,200000,1,1,0
4,93,4800000.0,50000,3,3,0


In [181]:
filtered_product_df.head()

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time
0,241,241,241,245,5,404,Pending,230400,1200,2015-03-12,2015-02-11 21:00:01
4,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01
63,518,518,518,526,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04
83,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02
112,435,435,435,443,19,16378,Pending,0,200000,2015-08-14,2015-05-11 05:00:04


In [182]:
product_df_new=pd.merge(filtered_product_df,temp_product_df,on=['opportunity id'],how='inner')

In [183]:
product_df_new

,unnamed 0,unnamed 0,unnamed 0.2,unnamed 0.1,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won
0,241,241,241,245,5,404,Pending,230400,1200,2015-03-12,2015-02-11 21:00:01,230400.0,1200,1,1,0
1,266,266,266,272,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200,1,1,0
2,518,518,518,526,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04,0.0,1800000,5,5,0
3,447,447,447,455,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
4,435,435,435,443,19,16378,Pending,0,200000,2015-08-14,2015-05-11 05:00:04,0.0,1800000,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9369,135452,135452,135500,136672,42781,38151,Pending,75600,25200,2016-11-16,2015-11-15 05:00:02,75600.0,25200,1,1,0
9370,139714,139714,139762,140934,42789,38158,Pending,60000,10000,2016-09-01,2015-11-15 13:00:03,33000.0,11000,2,2,0
9371,139711,139711,139759,140931,42789,38158,Pending,6000,1000,2016-09-01,2015-11-16 13:00:03,33000.0,11000,2,2,0
9372,105065,105065,105113,106285,56117,25835,Pending,7605,65,2015-08-28,2015-07-24 05:00:03,7605.0,65,1,1,0


In [186]:
product_df_new.drop(columns=['unnamed 0','unnamed 0','unnamed 0.1','unnamed 0.2'
],inplace=True)


In [188]:
oppty_df_new.drop(columns=['unnamed 0','unnamed 0','unnamed 0.1'
],inplace=True)

In [190]:
oppty_df_new.drop(columns=['unnamed 0.2'
],inplace=True)

In [191]:
oppty_df_new


,opportunity id,transition to stage,transition to timestamp,transition from stage name,transition from timestamp,customer name,risk status,creation date,decision date,win,opportunity type,core consumption market,core product segment,core sales segment,geography,core product application,no of days to decision,transition days,total transition days
0,5,3,2015-02-11 13:00:01,0,NaT,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,NaN,30.0
1,5,3,2015-02-13 05:00:02,3,2015-02-11 13:00:01,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,1.0,30.0
2,5,3,2015-02-18 21:00:07,3,2015-02-13 05:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,5.0,30.0
3,5,3,2015-03-04 21:00:02,3,2015-02-18 21:00:07,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,13.0,30.0
4,5,3,2015-03-04 21:00:02,3,2015-03-04 21:00:02,83,None,2015-02-11,2015-03-12,Closed/Won,None,11,3,8,-1,4,29,0.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87556,42789,6,2016-06-01 05:00:03,6,2016-05-25 07:21:17,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,6.0,64.0
87557,42789,6,2016-06-06 08:05:32,6,2016-06-01 05:00:03,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,5.0,64.0
87558,42789,6,2016-06-10 07:13:44,6,2016-06-06 08:05:32,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,3.0,64.0
87559,42789,1,2016-06-12 21:00:04,6,2016-06-10 07:13:44,9787,None,2015-11-15,2016-09-01,Closed/Lost,Design,16,5,4,2,5,291,2.0,64.0


In [193]:
dataset.save_dataset(context, oppty_df_new, '/cleaned/oppty_data', index = False)
dataset.save_dataset(context, product_df_new, '/cleaned/prod_data', index = False)

In [196]:
product_df_new.head()

,opportunity id,product id,product status,product $,product quantity,decision date,snapshot time,opportunity average price,opportunity total quantity,no of products,no of products lost,no of products won
0,5,404,Pending,230400,1200,2015-03-12,2015-02-11 21:00:01,230400.0,1200,1,1,0
1,5,404,Win Approved,230400,1200,2015-03-16,2015-03-16 13:00:01,230400.0,1200,1,1,0
2,19,16377,Pending,0,400000,2015-08-14,2015-05-11 05:00:04,0.0,1800000,5,5,0
3,19,16377,Win Approved,0,400000,2015-08-18,2015-08-18 13:00:02,0.0,1800000,5,5,0
4,19,16378,Pending,0,200000,2015-08-14,2015-05-11 05:00:04,0.0,1800000,5,5,0


In [195]:
product_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 0 to 9373
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   opportunity id              9374 non-null   int64  
 1   product id                  9374 non-null   int64  
 2   product status              9374 non-null   object 
 3   product $                   9374 non-null   object 
 4   product quantity            9374 non-null   object 
 5   decision date               9374 non-null   object 
 6   snapshot time               9374 non-null   object 
 7   opportunity average price   9374 non-null   float64
 8   opportunity total quantity  9374 non-null   int64  
 9   no of products              9374 non-null   int64  
 10  no of products lost         9374 non-null   int64  
 11  no of products won          9374 non-null   int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 952.0+ KB


In [199]:
columns_to_convert = [
    'opportunity id', 'product id', 'product $',
    'product quantity', 'opportunity total quantity',
    'no of products', 'no of products lost', 'no of products won'
]

# Convert the selected columns to int
for column in columns_to_convert:
    product_df_new[column] = product_df_new[column].astype(int)

product_df_new['decision date'] = pd.to_datetime(product_df_new['decision date'], format = '%Y-%m-%d')

In [203]:
#merging of datasets
merged_df=pd.merge(oppty_df_new,product_df_new,how='inner',left_on=['opportunity id','decision date'],right_on=['opportunity id','decision date'])
dataset.save_dataset(context,merged_df , 'processed/merged')

ValueError: Invalid dataset key: processed/merged. 

Available datasets: ['/raw/sod', '/raw/spd', '/cleaned/oppty_data', '/cleaned/prod_data']

In [ ]:
merged_df.columns

In [ ]:
opportunity_df.columns

In [ ]:
market_dummy=pd.crosstab(opportunity_df['core consumption market'],opportunity_df['opportunity status'])
market_dummy

In [ ]:
market_dummy=market_dummy[market_dummy['Closed/Won']>0]
market_dummy['won_percent']=market_dummy['Closed/Won']/(market_dummy['Closed/Won'].sum())
market_dummy=market_dummy.sort_values('won_percent',ascending=False)
# Create the stacked bar plot
plt.figure(figsize=(10, 6))
market_dummy[['won_percent']].plot(kind='bar', figsize=(10, 6))
plt.title('Proportion of won by core consumption market')
plt.xlabel('Core consumption markets')
plt.ylabel('Proportion')
plt.show()

In [ ]:
segment_dummy=pd.crosstab(opportunity_df['core product segment'],opportunity_df['opportunity status'])
segment_dummy

In [ ]:
segment_dummy=segment_dummy[segment_dummy['Closed/Won']>0]
segment_dummy['won_percent']=segment_dummy['Closed/Won']/(segment_dummy['Closed/Won'].sum())
segment_dummy=segment_dummy.sort_values('won_percent',ascending=False)
# Create the stacked bar plot
plt.figure(figsize=(10, 6))
segment_dummy[['won_percent']].plot(kind='bar', figsize=(10, 6))
plt.title('Proportion of won by core product segments')
plt.xlabel('Core product segments')
plt.ylabel('Proportion')
plt.show()

In [ ]:
sales_dummy=pd.crosstab(opportunity_df['core sales segment'],opportunity_df['opportunity status'])
sales_dummy

In [ ]:
sales_dummy=sales_dummy[sales_dummy['Closed/Won']>0]
sales_dummy['won_percent']=sales_dummy['Closed/Won']/(sales_dummy['Closed/Won'].sum())
sales_dummy=sales_dummy.sort_values('won_percent',ascending=False)
# Create the stacked bar plot
plt.figure(figsize=(10, 6))
sales_dummy[['won_percent']].plot(kind='bar', figsize=(10, 6))
plt.title('Proportion of won by geography')
plt.xlabel('geographies')
plt.ylabel('Proportion')
plt.show()

In [ ]:
geography_dummy=pd.crosstab(opportunity_df['geography'],opportunity_df['opportunity status'])
geography_dummy

In [ ]:
geography_dummy=geography_dummy[geography_dummy['Closed/Won']>0]
geography_dummy['won_percent']=geography_dummy['Closed/Won']/(geography_dummy['Closed/Won'].sum())
geography_dummy=geography_dummy.sort_values('won_percent',ascending=False)
# Create the stacked bar plot
plt.figure(figsize=(10, 6))
geography_dummy[['won_percent']].plot(kind='bar', figsize=(10, 6))
plt.title('Proportion of won by geography')
plt.xlabel('geographies')
plt.ylabel('Proportion')
plt.show()

In [ ]:
merged_df.columns

In [ ]:
transition_dummy=merged_df[['win','total transition days']]
transition_dummy

In [ ]:
transition_dummy=transition_dummy.drop_duplicates()
transition_dummy

In [ ]:
transition_dummy=transition_dummy.sort_values(by='total transition days')
transition_dummy

In [ ]:
transition_dummy=transition_dummy[transition_dummy['total transition days']>=0]
transition_dummy=transition_dummy.reset_index(drop=True)

In [ ]:
# Define the bins for 'total transition days'
bins = [0, 50, 100, 200, 300, 400, 500,600]
labels = ['0-50', '51-100', '101-200', '201-300', '301-400', '401-500','501-600']

# Bin the 'total transition days' column
transition_dummy['binned_days'] = pd.cut(transition_dummy['total transition days'], bins=bins, labels=labels, right=False)

# Count the occurrences of each bin for each 'win' class
bin_win_counts = transition_dummy.groupby(['binned_days', 'win']).size().unstack(fill_value=0)

# Plot the side-by-side bar chart
ax = bin_win_counts.plot(kind='bar', color=['blue', 'orange'], figsize=(10, 6), width=0.8)
plt.xlabel('Total Transition Days')
plt.ylabel('Count')
plt.title('Count of Win Classes by Total Transition Days Bins')
plt.legend(title='Win', labels=['0', '1'])
plt.xticks(rotation=45)
plt.show()

In [7]:
a = data['pri_bpm_df'].query('pri_sales_amount < 0').groupby('customer_code').agg({'month_start_date':['min','max']})
b = data['pri_bpm_df'].query('pri_sales_amount > 0').groupby('customer_code').agg({'month_start_date':['min','max']})
a.columns = ['neg_min','neg_max']
b.columns = ['pos_min','pos_max']
c = a.merge(b, left_index=True, right_index=True)
print(c.shape, sum(c['neg_min']>c['pos_max']))

(174, 4) 174


This implies that in all the customers once the sales turn negative , they dont turn positive again. Thus inferring case 1

Reference Date (assumed and considered) :: 2013-05-01

In [8]:
ref_date_for_churn = r'2013-05-01'

#### Customer Population

Exhaustive set of customer ids in the data to be considered for churn

 - Date range of the primary sales data is 2011-04 till 2013-12. Exhaustive list of customers prone to churn can be considered as the ones that have been active during 2011-04 till 2013-12.
 - Also only those customers whose date_of_joining is before the reference date are to be considered.

In [9]:
df_population = data['pri_bpm_df'][['customer_code']].drop_duplicates() \
                    .merge(data['doj_df'], on='customer_code', how='left', validate='1:1') \
                    .query(f"date_of_joining <= '{ref_date_for_churn}'or date_of_joining.isnull()",engine='python')[['customer_code']]
df_population.drop_duplicates(inplace=True)

#### Ref-Date Based Decision Variable Generation for Churn.
 - No sales post a date is considered customer churned.
 - Ref date is used to distinguish between customer about to churn in the future to the customers that have already churned in the past.

In [10]:
# obtaining the min month start date when the sales become 0 or negative and considering that month as attrition month
cust = data['pri_bpm_df'] \
           .query('pri_sales_amount <= 0') \
           .groupby('customer_code')['month_start_date'].min() \
           .rename('attrition_month_strt') \
           .reset_index()

cust['ref_date'] = pd.to_datetime(ref_date_for_churn)

# obtaining attrition month - ref month days
cust['days_from_ref_date_to_attrition'] = (cust.attrition_month_strt - cust.ref_date).dt.days
cust['target'] = 1
df_population = df_population.merge(cust, on='customer_code', how='left')

# filtering out customers who have already been churned before the reference date
df_population = df_population[(df_population.days_from_ref_date_to_attrition>=0) | (df_population.days_from_ref_date_to_attrition.isnull())]

df_population = df_population.drop(['ref_date', 'days_from_ref_date_to_attrition', 'attrition_month_strt'], axis=1)
df_population['target'].fillna(0, inplace=True)
df_population['ref_date']=pd.to_datetime(ref_date_for_churn)
display_as_tabs([('target summary',df_population.target.describe()),('head',df_population.head()), ('tail',df_population.tail())])


Tabs
    [0] DataFrame(Series, name='target summary')
    [1] DataFrame(name='head', value=   customer_code  ...)
    [2] DataFrame(name='tail', value=     customer_code  ...)

### Basic Features

Most of the features here are derived from the data sources and are to be aggregated to be at customer level. Basic customer level features are added here. Rest are generated as part of feature generation

### Customer area code

In [11]:
df_customer_area_code = pd.DataFrame()
for key_ in ['pri_bpm_df','coverage_df','retail_program_df','ec_df',
             'ordered_with_app_df','ordered_without_app_df','dist_retail_invoice_df']:
    df_customer_area_code = pd.concat([
        df_customer_area_code,
        data[key_]
            .filter(['customer_code','asm_area_code'])
            .drop_duplicates()
    ]).drop_duplicates()
   
df_population = df_population.merge(
    df_customer_area_code,
    on = 'customer_code',
    how = 'left',
    validate = '1:1'
)

### Date joined

In [12]:
df_population = df_population.merge(
    data['doj_df'],
    on = 'customer_code',
    how = 'left',
    validate = '1:1'
)

**DEV Notes**

<details>
    
    1. validate='1:1' : This ensures extra rows will not be created even by accident.
    2. For more merge rules refer guidelines [here]https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)
    3. More information on a merge can be obtained by using the ta_lib.core.api.merge_info function

## Test Train Split.

In [13]:
from sklearn.model_selection import train_test_split

df_population.set_index('customer_code',inplace=True)
train_X, test_X, train_y, test_y = train_test_split(
    df_population.drop('target',axis=1), df_population[['target']], test_size=0.2, random_state=context.random_seed)

train_X = train_X.reset_index()
train_y = train_y.reset_index()
test_X = test_X.reset_index()
test_y = test_y.reset_index()

print(f"Saving training datasets")
dataset.save_dataset(context, train_X, 'train/attrition/features')
dataset.save_dataset(context, train_y, 'train/attrition/target')

print(f"Saving test datasets")
dataset.save_dataset(context, test_X, 'test/attrition/features')
dataset.save_dataset(context, test_y, 'test/attrition/target')

Saving training datasets
Saving test datasets
